In [3]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                                AutoMinorLocator)

import warnings
import arviz as az
import numpy as np
import pymc3 as pm
import theano.tensor as tt                                       
from scipy.integrate import solve_ivp
from scipy.stats import norm      
import numpy as numpy


In [4]:
weeks = mdates.WeekdayLocator()
years = mdates.YearLocator()
months = mdates.MonthLocator()
weekdays = mdates.DayLocator()
dateFmt = mdates.DateFormatter('%b-%y')
dateFmt_day = mdates.DateFormatter('%d-%b')


def rolling_avg_df(dfs, N=7):
     # Define a rolling average function
     dfs = dfs.loc[:].rolling(N, win_type="boxcar").mean()
     return dfs

query = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=nation&areaName={}&metric=hospitalCases&metric=newAdmissions&metric=newCasesBySpecimenDate&metric=newDeaths28DaysByDeathDate&format=csv'
area = 'England'
df = pd.read_csv(query.format(area))
df.set_index(pd.to_datetime(df['date']),inplace=True)
df.sort_index(inplace=True)

#truncate in dates if required
df = df.truncate(before=dt.datetime(2020,7,1),after=dt.datetime(2020,12,31))
# df.truncate(before=dt.datetime(2020,1,8),after=dt.datetime(2020,12,31))

# fig, ax = plt.subplots(figsize=(12,8),sharex= True, facecolor='white')
# ax.plot(df.index,rolling_avg_df(df['newCasesBySpecimenDate']), label = 'New Cases') # daily number of new cases
# # ax.plot(df.index,rolling_avg_df(df['newAdmissions']), label = 'new admissions') # daily number of new admissions to hospital of patients with COVID-19
# ax.plot(df.index,rolling_avg_df(df['hospitalCases']), label = 'hospital cases') # daily number of confirmed COVID-19 patients in hospital
# # ax.plot(df.index,rolling_avg_df(df['newDeaths28DaysByDeathDate']), label = 'deaths') # Daily numbers of people who died within 28 days of being identified as a COVID-19 case by a positive test. Data are shown by date of death.

# # save data for daily number of new cases




# ax.legend()
# ax.grid(which='both')
# ax.set_xlabel('Date')
# ax.set_ylabel('Counts')
# # ax.set_yscale('log')
# #ax.set_title('{} Cases'.format(reg))

# # format the ticks
# month_dateFmt = mdates.DateFormatter('%b')
# ax.xaxis.set_major_locator(months)
# ax.xaxis.set_major_formatter(dateFmt)
# ax.xaxis.set_minor_locator(weeks)

# #ax.xaxis.set_major_locator(weeks)
# #ax.xaxis.set_major_formatter(dateFmt_day)
# #ax.xaxis.set_minor_locator(weekdays)

# ax.tick_params(axis="both", direction="in", which="both", 
# right=True,left=True, top=True, bottom=True)

# #ax.legend(ncol=1, title='')
# _= fig.autofmt_xdate()

# plt.tight_layout()


In [5]:
def sir_odes(t, x, b, g, N):
    "SIR Model"
    

    S = x[0]
    I = x[1]
    R = x[2]


    dSdt = -(b/N)*S*I
    dIdt = (b/N)*S*I - g*I
    dRdt = g*I
    case = (b/N)*S*I
   
    
    return dSdt, dIdt, dRdt


In [6]:
new_cases = rolling_avg_df(df['newCasesBySpecimenDate'])
new_cases =  new_cases.to_numpy()
new_cases = np.array(new_cases[6:])


In [9]:
"Set up of Model parameters"
lc = len(new_cases)
t_span = np.array([1, lc])  # Time limits
t = np.linspace(t_span[0], t_span[1], t_span[1] ) # Time series, but want to only sample for period 1: 115
x_0 = np.array([54000000 - new_cases[0] , new_cases[0], 0])  # Initial conditions for model variables: S, I, R respectively
N = np.sum(x_0)
gen = 10000
data = new_cases # Daily new cases
print(54000000-new_cases[0])

53999448.428571425


In [9]:
## LogLikelihood and gradient of the LogLikelihood functions
def log_likelihood(solve_ivp, sir_odes, t_span, x_0, theta , t,  data):


    sol = solve_ivp(sir_odes, t_span , x_0, args = theta , t_eval = t)
    
    S = sol.y[0]
    I = sol.y[1]
    
    I = (np.multiply(S,I)*theta[0]) / theta[2]
    # Used pandas to do truncated calcl, evolve for same number of days for the whole dfata set, including all missing variables , evolve for the same period, insert as 
    # isnert as a new column, pandas drop nans, uses pandas to truncate,insert the model values into pandas, insert I as a new column in pandas
    # Easier to cut out of model in pandas, we tell pandas, i.e drop Nans. 
    # Dropping days that match in data and model. Then data - I would do it daya by day, we need a day by day match.  
    logp = -0.5*np.sum(np.square((data[0:125] - I[0:125])))#/(np.std((data))**2)
#     I = sol.y[1]
    
#     logp  = -0.5*np.sum(np.square((data-I)))/(np.std((data))**2)
    return logp


## Wrapper classes to theano-ize LogLklhood and gradient...
class Loglike(tt.Op):
    itypes = [tt.dvector]
    otypes = [tt.dscalar]

    def __init__(self, solve_ivp, sir_odes, t_span,x_0, t, data):
        self.data = data
        self.solve_ivp = solve_ivp
        self.sir_odes = sir_odes
        self.t_span = t_span
        self.x_0 = x_0
        self.t = t
        # self.loglike_grad = LoglikeGrad(self.data, self.t)

    def perform(self, node, inputs, outputs):
        (theta,) = inputs 
        logp = log_likelihood(self.solve_ivp,self.sir_odes, self.t_span,self.x_0, theta , self.t, self.data)
        
        outputs[0][0] = np.array(logp)


chi_store = []
loglike = Loglike(solve_ivp, sir_odes, t_span,x_0, t, data)
with pm.Model() as model:
    sigma = 0.5 #pm.HalfNormal('sigma' ,sd = 10)

    # b_0 = pm.Normal('b_0', mu = 5, sigma= 10)
    b_0 = pm.Uniform('b_0',lower = 0, upper = 5)
     # split beta into five seperate paramers, then add an if statment into the integrator, then say if t = 5 then use this beta
    # g_0 = pm.Normal('g_0', mu = 5 , sigma= 10)
    g_0 = pm.Uniform('g_0', lower = 0, upper = 5)
    
    
    theta = tt.as_tensor_variable([b_0,g_0,N])
    
    pm.Potential("like", loglike(theta))
    



with model:
    trace = pm.sample(draws = 100, chains = 1, tune = 10, cores  = 1)
    # trace = pm.sample(step=pm.NUTS())
    # idata = pm.sample(return_inferencedata=True)
    print(pm.summary(trace).to_string())

C:\Users\sulai\AppData\Local\Temp/ipykernel_19096/1245208205.py:63: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(draws = 100, chains = 1, tune = 10, cores  = 1)
Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS failed. Falling back to elementwise auto-assignment.
Sequential sampling (1 chains in 1 job)
CompoundStep
>Slice: [g_0]
>Slice: [b_0]


Sampling 1 chain for 10 tune and 100 draw iterations (10 + 100 draws total) took 8 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks
arviz - WARNING - Shape validation failed: input_shape: (1, 100), minimum_shape: (chains=2, draws=4)


      mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  r_hat
b_0  2.521  0.001   2.520    2.523      0.001    0.001       1.0      13.0    NaN
g_0  2.490  0.001   2.488    2.492      0.001    0.001       1.0      13.0    NaN


In [10]:
b_0 = trace["b_0"]
g_0 = trace["g_0"]

chi_store = []
for b_0, g_0 in zip(b_0, g_0 ):
    
#     args = (b_a, b_sy, g_a, g_sy, e_a, e_sy, a, N) 
#     sol = solve_ivp(seasyr_odes, t_span, x_0, args=(args), t_eval=t)
    
#     E = sol.y[1]
    
    chi_value = 1
    chi_store.append(chi_value)
    
chains = np.column_stack([ chi_store, trace["b_0"] ,trace["g_0"]])
print(chains)
datafile_path = "chain_SIR_con"
np.savetxt(datafile_path , chains, fmt=['%f','%f','%f'])

[[1.         2.52308597 2.49199622]
 [1.         2.52307581 2.49193128]
 [1.         2.52305565 2.49192961]
 [1.         2.52300386 2.49190314]
 [1.         2.52298834 2.49185334]
 [1.         2.52297462 2.49185085]
 [1.         2.5229682  2.49181246]
 [1.         2.5229297  2.49182918]
 [1.         2.52283548 2.4917693 ]
 [1.         2.52276375 2.49164296]
 [1.         2.52273248 2.49159935]
 [1.         2.52268981 2.49158825]
 [1.         2.52269301 2.49158664]
 [1.         2.52264305 2.49152156]
 [1.         2.52261338 2.49149786]
 [1.         2.52258375 2.49147213]
 [1.         2.52252598 2.49144848]
 [1.         2.52240156 2.49133649]
 [1.         2.52239298 2.49122407]
 [1.         2.52237468 2.49126204]
 [1.         2.5223428  2.49122968]
 [1.         2.52231791 2.49117149]
 [1.         2.52228536 2.49117105]
 [1.         2.52228396 2.49116809]
 [1.         2.52227549 2.49115542]
 [1.         2.52226065 2.49112926]
 [1.         2.52225203 2.49112526]
 [1.         2.52218799 2.49

In [ ]:
t_span = np.array([1, lc])  # Time limits
t = np.linspace(t_span[0], t_span[1], t_span[1] ) # Time series, but want to only sample for period 1: 115
x_0 = np.array([54000000 - new_cases[0] , new_cases[0], 0])  # Initial conditions for model variables: S, I, R respectively
N = np.sum(x_0)
# args = [2.386, 2.355, N]
args = [2.394, 2.362, N]
sol = solve_ivp(sir_odes, t_span, x_0, args=args, t_eval=t)
S = sol.y[0]
I = sol.y[1]
I = (np.multiply(S,I)*args[0]) / args[2]
# plt.plot(t, I, label="Model")
# plt.plot(t, data, label="Data")
# plt.legend()
# # truncate to day 60 to catagorise the peak. 
# # Add an r that is binwise overa certain periods, this is r_t
# # rough thing to quantify,, calcaulte loglike / number of days, look for goodness of fit statistic, want to comapre this to antoher model with a different parameter set
# plt.show()
dates1 = pd.date_range(start='07/07/2020', end='31/12/2020', periods = 178)
plt.figure(figsize=(15,9))
# plt.plot(dates1, S)
plt.plot(dates1,data,label = 'Daily new infection cases',color='teal')
plt.plot(dates1[0:126],I[0:126],label = r'Model daily new  infection cases', color = 'r')
plt.plot(dates1[125:],I[125:],label = r'Predicted daily new  infection cases', color = 'b', ls = '--')
plt.xlabel("Date", fontsize = 17)
plt.ylabel("Population", fontsize = 17)
plt.axvline(dates1[125:126], c = 'm',ls = '--', label = 'Cut-Off')
plt.legend(fontsize=13)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.grid()
plt.savefig('SIR,,pred')
plt.show()





# plt.plot(t,I_data/N,label = 'Infected Data',color='teal')
# plt.plot(t,I/N,label = 'Infected Signal',color='black')
# plt.plot(t,H_data/N,label = 'Hospitalised Data',color='green')
# plt.plot(t,H/N,label = 'Hospitalised Signal',color='m')

# plt.xlabel("Time[Days]", fontsize = 15)
# plt.ylabel("Fraction of Population", fontsize = 15)


# plt.grid()